In [1]:
import azure.cognitiveservices.speech as speechsdk
import time
import datetime
import openai
# Creates an instance of a speech config with specified subscription key and service region.
# Replace with your own subscription key and region identifier from here: https://aka.ms/speech/sdkregion
speech_key, service_region = "125171d59a4b4fda87649a90af673010", "EastUS"
speech_config = speechsdk.SpeechConfig(subscription=speech_key, region=service_region)

# Creates an audio configuration that points to an audio file.
# Replace with your own audio filename.
audio_filename = "BillGates_2010.wav"
audio_input = speechsdk.audio.AudioConfig(filename=audio_filename)

# Creates a recognizer with the given settings
speech_config.speech_recognition_language="en-US"
speech_config.request_word_level_timestamps()
speech_config.enable_dictation()
speech_config.output_format = speechsdk.OutputFormat(1)

speech_recognizer = speechsdk.SpeechRecognizer(speech_config=speech_config, audio_config=audio_input)

#result = speech_recognizer.recognize_once()
all_results = []



#https://learn.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.recognitionresult?view=azure-python
def handle_final_result(evt):
    all_results.append(evt.result.text) 
    
    
done = False

def stop_cb(evt):
    print('CLOSING on {}'.format(evt))
    speech_recognizer.stop_continuous_recognition()
    global done
    done= True

#Appends the recognized text to the all_results variable. 
speech_recognizer.recognized.connect(handle_final_result) 

#Connect callbacks to the events fired by the speech recognizer & displays the info/status
#Ref:https://learn.microsoft.com/en-us/python/api/azure-cognitiveservices-speech/azure.cognitiveservices.speech.eventsignal?view=azure-python   
speech_recognizer.recognized.connect(lambda evt: print('RECOGNIZED: {}'.format(evt)))
speech_recognizer.session_started.connect(lambda evt: print('SESSION STARTED: {}'.format(evt)))
speech_recognizer.session_stopped.connect(lambda evt: print('SESSION STOPPED {}'.format(evt)))
speech_recognizer.canceled.connect(lambda evt: print('CANCELED {}'.format(evt)))
# stop continuous recognition on either session stopped or canceled events
speech_recognizer.session_stopped.connect(stop_cb)
speech_recognizer.canceled.connect(stop_cb)

speech_recognizer.start_continuous_recognition()

while not done:
    time.sleep(.5)
    
results = ""
results = " ".join(all_results).strip()

openai.api_key = "sk-proj-x4PXDZxCezGbkeYUWid9T3BlbkFJhqY6m6CLcL8TpvrMvzTu"

# create a chat completion
chat_completion = openai.ChatCompletion.create(model="gpt-4-turbo", messages=[{"role": "system", "content": "You will summarize the following speech from bill gates"},
                                                                              {"role": "user", "content": results}])
# print the chat completion
print("\n\n")
print("chatgpt summary=====================================================================================================================")
print(chat_completion.choices[0].message.content)


SESSION STARTED: SessionEventArgs(session_id=2f90279709ec41e490fcfd28d3334115)
RECOGNIZED: SpeechRecognitionEventArgs(session_id=2f90279709ec41e490fcfd28d3334115, result=SpeechRecognitionResult(result_id=188516671721407ab0e07574108d2b03, text="", reason=ResultReason.RecognizedSpeech))
RECOGNIZED: SpeechRecognitionEventArgs(session_id=2f90279709ec41e490fcfd28d3334115, result=SpeechRecognitionResult(result_id=5f505300d8424aea82946a2e0dbcfcab, text="I'm going to talk today about energy and climate and that might seem a bit surprising because my full time work at the foundation is mostly about vaccines and seeds about the things that we need to invent and deliver to help the poorest 2 billion live better lives but energy and climate are extremely important to these people in fact more important than to anyone else on the planet", reason=ResultReason.RecognizedSpeech))
RECOGNIZED: SpeechRecognitionEventArgs(session_id=2f90279709ec41e490fcfd28d3334115, result=SpeechRecognitionResult(result_i

In [3]:
print("\n\n")
print("chatgpt summary=====================================================================================================================")
print(chat_completion.choices[0].message.content)




chatgpt summary=====================================================================================================================
Bill Gates discusses the intertwined issues of energy and climate change, emphasizing their critical impact on the world's poorest populations who are most vulnerable to environmental shifts impacting agriculture and stability. He points out that affordable energy is key to reducing poverty, highlighting historical reductions in energy costs due to technological advances. Despite the progress in making energy cheaper and more accessible, Gates identifies a major challenge: the need to reduce CO2 emissions to zero to avoid catastrophic global warming.

He explains the CO2 emissions formula, which includes population, services provided per person, energy used per service, and CO2 produced per unit of energy. Gates stresses the necessity for one of these variables to near zero, considering the persistent rise in global population and the increasing demand